In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
!ls "/content/drive/My Drive/Colab Notebooks/Adopt a Buddy/"

 clr_callback.py			'Pet Adoption (CB) - BC.ipynb'
'Copy of Pet Adoption (CB) - BC.ipynb'	'Pet Adoption (CB) - PC.ipynb'
'Copy of Pet Adoption (CB) - PC.ipynb'	'Pet Adoption (EDA).ipynb'
 Dataset				'Pet Adoption (LGB) - BC.ipynb'
 learningratefinder.py			'Pet Adoption (LGB) - PC.ipynb'
'NPZ Files'				 Predictions
'Pet Adoption (ANN) - BC.ipynb'		 __pycache__
'Pet Adoption (ANN) - PC.ipynb'


In [ ]:
! pip install catboost

     |████████████████████████████████| 65.9MB 45kB/s 


In [ ]:
! pip install optuna

     |████████████████████████████████| 235kB 12.8MB/s 
     |████████████████████████████████| 1.1MB 9.9MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 10.0MB/s 
     |████████████████████████████████| 81kB 11.8MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 112kB 49.1MB/s 
     |████████████████████████████████| 133kB 53.2MB/s 
  Created wheel for alembic: filename=alembic-1.4.2-cp36-none-any.whl size=159543 sha256=2c3846aed0ea58196543365de203065cbdde774150e885ca61497e299dacba31
  Stored in directory: /root/.cache/pip/wheels/1f/04/83/76023f7a4c14688c0b5c2682a96392cfdd3ee4449eaaa287ef
Successfully built alembic
  Created wheel for optuna: filename=optuna-2.0.0-cp36-none-any.whl size=312964 sha256=12446e8cb5374d662b5b8c5b28e49472d0d44ea95b57b9cf402e6c8d461e9c56
  Stored in directory: /root/.cache/pip/wheels/

## Import Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE, ADASYN
from catboost import CatBoostClassifier
import optuna

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


## Set file paths for train and predict datasets

In [ ]:
train_dataset = "/content/drive/My Drive/Colab Notebooks/Adopt a Buddy/Dataset/train.csv"
predict_dataset = "/content/drive/My Drive/Colab Notebooks/Adopt a Buddy/Dataset/test.csv"

## Data Preprocessing

### Read train and predict datasets

In [ ]:
train_df = pd.read_csv(train_dataset)
predict_df = pd.read_csv(predict_dataset)
print("train_df: {}".format(train_df.shape))
print("predict_df: {}".format(predict_df.shape))

train_df: (18834, 11)
predict_df: (8072, 9)


### Extract target variables into NumPy arrays

In [ ]:
train_df['pet_category'] = train_df['pet_category'].apply(lambda x: 3 if x==4 else x)
train_y_pc = train_df['pet_category']
train_df.drop(['breed_category','pet_category'], inplace=True, axis=1)
print("train_y_pc: {}".format(train_y_pc.shape))

train_y_pc: (18834,)


In [ ]:
temp_df = pd.DataFrame(train_y_pc.values, columns=['Class'])
temp_df.groupby(['Class']).size().reset_index().rename(columns={0:'count'})

,Class,count
0,0,88
1,1,7184
2,2,10621
3,3,941


### Combine train and predict dataframes

In [ ]:
combined_df = train_df.append(predict_df, sort=False, ignore_index=True)
combined_df.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,NaN,Brown,0.15,40.90,15,4
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4


### Feature Engineering

#### Issue Date and Listing Date

In [ ]:
combined_df['iss_dt_year'] = combined_df['issue_date'].map(lambda x: pd.to_datetime(x).year)
combined_df['iss_dt_quarter'] = combined_df['issue_date'].map(lambda x: pd.to_datetime(x).quarter)
combined_df['iss_dt_month'] = combined_df['issue_date'].map(lambda x: pd.to_datetime(x).month)
combined_df['iss_dt_week'] = combined_df['issue_date'].map(lambda x: pd.to_datetime(x).week)
combined_df['iss_dt_day_year'] = combined_df['issue_date'].map(lambda x: pd.to_datetime(x).dayofyear)
combined_df['iss_dt_day_month'] = combined_df['issue_date'].map(lambda x: pd.to_datetime(x).day)
combined_df['iss_dt_day_week'] = combined_df['issue_date'].map(lambda x: pd.to_datetime(x).dayofweek)
combined_df['iss_dt_day_weekend'] = np.where(combined_df['iss_dt_day_week'].isin([5,6]),1,0)

In [ ]:
combined_df['lst_dt_year'] = combined_df['listing_date'].map(lambda x: pd.to_datetime(x).year)
combined_df['lst_dt_quarter'] = combined_df['listing_date'].map(lambda x: pd.to_datetime(x).quarter)
combined_df['lst_dt_month'] = combined_df['listing_date'].map(lambda x: pd.to_datetime(x).month)
combined_df['lst_dt_week'] = combined_df['listing_date'].map(lambda x: pd.to_datetime(x).week)
combined_df['lst_dt_day_year'] = combined_df['listing_date'].map(lambda x: pd.to_datetime(x).dayofyear)
combined_df['lst_dt_day_month'] = combined_df['listing_date'].map(lambda x: pd.to_datetime(x).day)
combined_df['lst_dt_day_week'] = combined_df['listing_date'].map(lambda x: pd.to_datetime(x).dayofweek)
combined_df['lst_dt_day_weekend'] = np.where(combined_df['lst_dt_day_week'].isin([5,6]),1,0)
combined_df['lst_dt_hour'] = combined_df['listing_date'].map(lambda x: pd.to_datetime(x).hour)
combined_df['lst_dt_minute'] = combined_df['listing_date'].map(lambda x: pd.to_datetime(x).minute)
combined_df['lst_dt_minutes_elapsed'] = combined_df['listing_date'].map(lambda x: (pd.to_datetime(x) - pd.to_datetime(pd.to_datetime(x).date())).seconds / 60.0)

In [ ]:
combined_df['dt_diff'] = (pd.to_datetime(combined_df['listing_date']) - pd.to_datetime(combined_df['issue_date']))
combined_df['dt_diff_days'] = combined_df['dt_diff']/np.timedelta64(1,'D')
combined_df['dt_diff_weeks'] = combined_df['dt_diff']/np.timedelta64(1,'W')
combined_df['dt_diff_mnths'] = combined_df['dt_diff']/np.timedelta64(1,'M')
combined_df['dt_diff_yrs'] = combined_df['dt_diff']/np.timedelta64(1,'Y')
combined_df['dt_diff_sec'] = combined_df['dt_diff']/np.timedelta64(1,'s')
combined_df['dt_diff_min'] = combined_df['dt_diff']/np.timedelta64(1,'m')
combined_df['dt_diff_hrs'] = combined_df['dt_diff']/np.timedelta64(1,'h')

In [ ]:
combined_df['curr_iss_dt_diff'] = (pd.datetime.now() - pd.to_datetime(combined_df['issue_date']))
combined_df['curr_iss_dt_diff_mnths'] = combined_df['curr_iss_dt_diff']/np.timedelta64(1,'M')
combined_df['curr_iss_dt_diff_yrs'] = combined_df['curr_iss_dt_diff']/np.timedelta64(1,'Y')

combined_df['curr_lst_dt_diff'] = (pd.datetime.now() - pd.to_datetime(combined_df['listing_date']))
combined_df['curr_lst_dt_diff_mnths'] = combined_df['curr_lst_dt_diff']/np.timedelta64(1,'M')
combined_df['curr_lst_dt_diff_yrs'] = combined_df['curr_lst_dt_diff']/np.timedelta64(1,'Y')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.



In [ ]:
combined_df.drop(['issue_date', 'listing_date', 'dt_diff', 'curr_iss_dt_diff', 'curr_lst_dt_diff'], axis=1, inplace=True)
combined_df.head()

,pet_id,condition,color_type,length(m),height(cm),X1,X2,iss_dt_year,iss_dt_quarter,iss_dt_month,iss_dt_week,iss_dt_day_year,iss_dt_day_month,iss_dt_day_week,iss_dt_day_weekend,lst_dt_year,lst_dt_quarter,lst_dt_month,lst_dt_week,lst_dt_day_year,lst_dt_day_month,lst_dt_day_week,lst_dt_day_weekend,lst_dt_hour,lst_dt_minute,lst_dt_minutes_elapsed,dt_diff_days,dt_diff_weeks,dt_diff_mnths,dt_diff_yrs,dt_diff_sec,dt_diff_min,dt_diff_hrs,curr_iss_dt_diff_mnths,curr_iss_dt_diff_yrs,curr_lst_dt_diff_mnths,curr_lst_dt_diff_yrs
0,ANSL_69903,2.0,Brown Tabby,0.80,7.78,13,9,2016,3,7,27,192,10,6,1,2016,3,9,38,265,21,2,0,16,25,985.0,73.684028,10.526290,2.420880,0.201740,6366300.0,106105.0,1768.416667,49.409542,4.117462,46.988661,3.915722
1,ANSL_66892,1.0,White,0.72,14.19,13,9,2013,4,11,47,325,21,3,0,2018,4,12,52,361,27,3,0,17,47,1067.0,1862.740972,266.105853,61.200139,5.100012,160940820.0,2682347.0,44705.783333,81.015940,6.751328,19.815801,1.651317
2,ANSL_69750,NaN,Brown,0.15,40.90,15,4,2014,3,9,39,271,28,6,1,2016,4,10,42,293,19,2,0,8,24,504.0,752.350000,107.478571,24.718372,2.059864,65003040.0,1083384.0,18056.400000,70.798071,5.899839,46.079699,3.839975
3,ANSL_71623,1.0,White,0.62,17.82,0,1,2016,4,12,52,366,31,5,1,2019,1,1,4,25,25,4,0,18,30,1110.0,755.770833,107.967262,24.830763,2.069230,65298600.0,1088310.0,18138.500000,43.692792,3.641066,18.862029,1.571836
4,ANSL_57969,2.0,Black,0.50,11.06,18,4,2017,3,9,39,271,28,3,0,2017,4,11,46,323,19,6,1,9,38,578.0,52.401389,7.485913,1.721642,0.143470,4527480.0,75458.0,1257.633333,34.789118,2.899093,33.067477,2.755623


#### Condition

In [ ]:
combined_df['condition'] = combined_df['condition'].fillna(value=3)
combined_df['condition'] = combined_df['condition'].apply(lambda x: int(x))
combined_df['condition_odd_even'] = combined_df['condition'].apply(lambda x: 'EVEN' if x%2==0 else 'ODD')
combined_df['condition_odd_even_enc'] = combined_df['condition_odd_even'].factorize()[0]
combined_df.drop(['condition_odd_even'], axis=1, inplace=True)

In [ ]:
combined_df['condition*X1'] = np.cbrt(combined_df['condition'] * combined_df['X1'])
combined_df['condition*X2'] = combined_df['condition'] * combined_df['X2']

In [ ]:
combined_df['condition%X1'] = combined_df.apply(lambda row: 0 if row['condition']==0 else row['X1']%row['condition'], axis=1)
combined_df['condition%X2'] = combined_df.apply(lambda row: 0 if row['condition']==0 else row['X2']%row['condition'], axis=1)

#### Pet Id

In [ ]:
combined_df['pet_id_num'] = combined_df['pet_id'].apply(lambda x: x.split('_')[1]).astype(int)
combined_df.head()

,pet_id,condition,color_type,length(m),height(cm),X1,X2,iss_dt_year,iss_dt_quarter,iss_dt_month,iss_dt_week,iss_dt_day_year,iss_dt_day_month,iss_dt_day_week,iss_dt_day_weekend,lst_dt_year,lst_dt_quarter,lst_dt_month,lst_dt_week,lst_dt_day_year,lst_dt_day_month,lst_dt_day_week,lst_dt_day_weekend,lst_dt_hour,lst_dt_minute,lst_dt_minutes_elapsed,dt_diff_days,dt_diff_weeks,dt_diff_mnths,dt_diff_yrs,dt_diff_sec,dt_diff_min,dt_diff_hrs,curr_iss_dt_diff_mnths,curr_iss_dt_diff_yrs,curr_lst_dt_diff_mnths,curr_lst_dt_diff_yrs,condition_odd_even_enc,condition*X1,condition*X2,condition%X1,condition%X2,pet_id_num
0,ANSL_69903,2,Brown Tabby,0.80,7.78,13,9,2016,3,7,27,192,10,6,1,2016,3,9,38,265,21,2,0,16,25,985.0,73.684028,10.526290,2.420880,0.201740,6366300.0,106105.0,1768.416667,49.409542,4.117462,46.988661,3.915722,0,2.962496,18,1,1,69903
1,ANSL_66892,1,White,0.72,14.19,13,9,2013,4,11,47,325,21,3,0,2018,4,12,52,361,27,3,0,17,47,1067.0,1862.740972,266.105853,61.200139,5.100012,160940820.0,2682347.0,44705.783333,81.015940,6.751328,19.815801,1.651317,1,2.351335,9,0,0,66892
2,ANSL_69750,3,Brown,0.15,40.90,15,4,2014,3,9,39,271,28,6,1,2016,4,10,42,293,19,2,0,8,24,504.0,752.350000,107.478571,24.718372,2.059864,65003040.0,1083384.0,18056.400000,70.798071,5.899839,46.079699,3.839975,1,3.556893,12,0,1,69750
3,ANSL_71623,1,White,0.62,17.82,0,1,2016,4,12,52,366,31,5,1,2019,1,1,4,25,25,4,0,18,30,1110.0,755.770833,107.967262,24.830763,2.069230,65298600.0,1088310.0,18138.500000,43.692792,3.641066,18.862029,1.571836,1,0.000000,1,0,0,71623
4,ANSL_57969,2,Black,0.50,11.06,18,4,2017,3,9,39,271,28,3,0,2017,4,11,46,323,19,6,1,9,38,578.0,52.401389,7.485913,1.721642,0.143470,4527480.0,75458.0,1257.633333,34.789118,2.899093,33.067477,2.755623,0,3.301927,8,0,0,57969


In [ ]:
combined_df['pet_id_odd_even'] = combined_df['pet_id_num'].apply(lambda x: 'EVEN' if x%2==0 else 'ODD')
combined_df['pet_id_odd_even_enc'] = combined_df['pet_id_odd_even'].factorize()[0]
combined_df['petid%X1'] = combined_df.apply(lambda row: 0 if row['X1']==0 else row['pet_id_num']%row['X1'], axis=1)
combined_df['petid%X2'] = combined_df.apply(lambda row: 0 if row['X2']==0 else row['pet_id_num']%row['X2'], axis=1)
combined_df['petid%condition'] = combined_df.apply(lambda row: 0 if row['condition']==0 else row['pet_id_num']%row['condition'], axis=1)

In [ ]:
combined_df['pet_id_0'] = combined_df['pet_id_num'].apply(lambda x: int(str(x)[0]))
combined_df['pet_id_1'] = combined_df['pet_id_num'].apply(lambda x: int(str(x)[1]))
combined_df['pet_id_2'] = combined_df['pet_id_num'].apply(lambda x: int(str(x)[2]))
combined_df['pet_id_3'] = combined_df['pet_id_num'].apply(lambda x: int(str(x)[3]))
combined_df['pet_id_4'] = combined_df['pet_id_num'].apply(lambda x: int(str(x)[4]))

In [ ]:
combined_df.drop(['pet_id', 'pet_id_num', 'pet_id_odd_even'], axis=1, inplace=True)
combined_df.head()

,condition,color_type,length(m),height(cm),X1,X2,iss_dt_year,iss_dt_quarter,iss_dt_month,iss_dt_week,iss_dt_day_year,iss_dt_day_month,iss_dt_day_week,iss_dt_day_weekend,lst_dt_year,lst_dt_quarter,lst_dt_month,lst_dt_week,lst_dt_day_year,lst_dt_day_month,lst_dt_day_week,lst_dt_day_weekend,lst_dt_hour,lst_dt_minute,lst_dt_minutes_elapsed,dt_diff_days,dt_diff_weeks,dt_diff_mnths,dt_diff_yrs,dt_diff_sec,dt_diff_min,dt_diff_hrs,curr_iss_dt_diff_mnths,curr_iss_dt_diff_yrs,curr_lst_dt_diff_mnths,curr_lst_dt_diff_yrs,condition_odd_even_enc,condition*X1,condition*X2,condition%X1,condition%X2,pet_id_odd_even_enc,petid%X1,petid%X2,petid%condition,pet_id_0,pet_id_1,pet_id_2,pet_id_3,pet_id_4
0,2,Brown Tabby,0.80,7.78,13,9,2016,3,7,27,192,10,6,1,2016,3,9,38,265,21,2,0,16,25,985.0,73.684028,10.526290,2.420880,0.201740,6366300.0,106105.0,1768.416667,49.409542,4.117462,46.988661,3.915722,0,2.962496,18,1,1,0,2,0,1,6,9,9,0,3
1,1,White,0.72,14.19,13,9,2013,4,11,47,325,21,3,0,2018,4,12,52,361,27,3,0,17,47,1067.0,1862.740972,266.105853,61.200139,5.100012,160940820.0,2682347.0,44705.783333,81.015940,6.751328,19.815801,1.651317,1,2.351335,9,0,0,1,7,4,0,6,6,8,9,2
2,3,Brown,0.15,40.90,15,4,2014,3,9,39,271,28,6,1,2016,4,10,42,293,19,2,0,8,24,504.0,752.350000,107.478571,24.718372,2.059864,65003040.0,1083384.0,18056.400000,70.798071,5.899839,46.079699,3.839975,1,3.556893,12,0,1,1,0,2,0,6,9,7,5,0
3,1,White,0.62,17.82,0,1,2016,4,12,52,366,31,5,1,2019,1,1,4,25,25,4,0,18,30,1110.0,755.770833,107.967262,24.830763,2.069230,65298600.0,1088310.0,18138.500000,43.692792,3.641066,18.862029,1.571836,1,0.000000,1,0,0,0,0,0,0,7,1,6,2,3
4,2,Black,0.50,11.06,18,4,2017,3,9,39,271,28,3,0,2017,4,11,46,323,19,6,1,9,38,578.0,52.401389,7.485913,1.721642,0.143470,4527480.0,75458.0,1257.633333,34.789118,2.899093,33.067477,2.755623,0,3.301927,8,0,0,0,9,1,1,5,7,9,6,9


#### Length and Height

In [ ]:
combined_df['height(m)'] = combined_df['height(cm)']/100

In [ ]:
combined_df['length_scale_10'] = np.array(np.round((combined_df['length(m)'] * 10)), dtype='int')
combined_df['length_scale_100'] = np.array(np.round((combined_df['length(m)'] * 100)), dtype='int')

In [ ]:
combined_df['height_scale_10'] = np.array(np.round((combined_df['height(m)'] * 10)), dtype='int')
combined_df['height_scale_100'] = np.array(np.round((combined_df['height(m)'] * 100)), dtype='int')
combined_df['height_scale_1000'] = np.array(np.round((combined_df['height(m)'] * 1000)), dtype='int')

In [ ]:
combined_df['diag(cm)'] = np.sqrt(combined_df['length(m)']**2 + combined_df['height(m)']**2)*100
combined_df['cbrt(m)'] = np.cbrt(combined_df['length(m)']**2 + combined_df['height(m)']**2)
combined_df['area(m)'] = np.sqrt(combined_df['length(m)'] * combined_df['height(m)'])

In [ ]:
combined_df['l_vs_h'] = combined_df.apply(lambda row: 'l=h' if np.round(row['length(m)'],1)==np.round(row['height(m)'],1) else 
                                          'l>h' if np.round(row['length(m)'],1)>np.round(row['height(m)'],1) else 
                                          'l<h' if np.round(row['length(m)'],1)<np.round(row['height(m)'],1) else 'NA', axis=1)
combined_df['l_vs_h_enc'] = combined_df['l_vs_h'].factorize()[0]

In [ ]:
combined_df['pet_size'] = combined_df['diag(cm)'].apply(lambda x: 'SMALL' if x<=40 else 'MEDIUM' if x>40 and x<=80 else 'BIG')
combined_df['pet_size_enc'] = combined_df['pet_size'].factorize()[0]

In [ ]:
combined_df.drop(['height(cm)', 'l_vs_h', 'pet_size'], axis=1, inplace=True)
combined_df.head()

,condition,color_type,length(m),X1,X2,iss_dt_year,iss_dt_quarter,iss_dt_month,iss_dt_week,iss_dt_day_year,iss_dt_day_month,iss_dt_day_week,iss_dt_day_weekend,lst_dt_year,lst_dt_quarter,lst_dt_month,lst_dt_week,lst_dt_day_year,lst_dt_day_month,lst_dt_day_week,lst_dt_day_weekend,lst_dt_hour,lst_dt_minute,lst_dt_minutes_elapsed,dt_diff_days,dt_diff_weeks,dt_diff_mnths,dt_diff_yrs,dt_diff_sec,dt_diff_min,dt_diff_hrs,curr_iss_dt_diff_mnths,curr_iss_dt_diff_yrs,curr_lst_dt_diff_mnths,curr_lst_dt_diff_yrs,condition_odd_even_enc,condition*X1,condition*X2,condition%X1,condition%X2,pet_id_odd_even_enc,petid%X1,petid%X2,petid%condition,pet_id_0,pet_id_1,pet_id_2,pet_id_3,pet_id_4,height(m),length_scale_10,length_scale_100,height_scale_10,height_scale_100,height_scale_1000,diag(cm),cbrt(m),area(m),l_vs_h_enc,pet_size_enc
0,2,Brown Tabby,0.80,13,9,2016,3,7,27,192,10,6,1,2016,3,9,38,265,21,2,0,16,25,985.0,73.684028,10.526290,2.420880,0.201740,6366300.0,106105.0,1768.416667,49.409542,4.117462,46.988661,3.915722,0,2.962496,18,1,1,0,2,0,1,6,9,9,0,3,0.0778,8,80,1,8,78,80.377412,0.864482,0.249479,0,0
1,1,White,0.72,13,9,2013,4,11,47,325,21,3,0,2018,4,12,52,361,27,3,0,17,47,1067.0,1862.740972,266.105853,61.200139,5.100012,160940820.0,2682347.0,44705.783333,81.015940,6.751328,19.815801,1.651317,1,2.351335,9,0,0,1,7,4,0,6,6,8,9,2,0.1419,7,72,1,14,142,73.384986,0.813589,0.319637,0,1
2,3,Brown,0.15,15,4,2014,3,9,39,271,28,6,1,2016,4,10,42,293,19,2,0,8,24,504.0,752.350000,107.478571,24.718372,2.059864,65003040.0,1083384.0,18056.400000,70.798071,5.899839,46.079699,3.839975,1,3.556893,12,0,1,1,0,2,0,6,9,7,5,0,0.4090,2,15,4,41,409,43.563861,0.574669,0.247689,1,1
3,1,White,0.62,0,1,2016,4,12,52,366,31,5,1,2019,1,1,4,25,25,4,0,18,30,1110.0,755.770833,107.967262,24.830763,2.069230,65298600.0,1088310.0,18138.500000,43.692792,3.641066,18.862029,1.571836,1,0.000000,1,0,0,0,0,0,0,7,1,6,2,3,0.1782,6,62,2,18,178,64.510095,0.746595,0.332391,0,1
4,2,Black,0.50,18,4,2017,3,9,39,271,28,3,0,2017,4,11,46,323,19,6,1,9,38,578.0,52.401389,7.485913,1.721642,0.143470,4527480.0,75458.0,1257.633333,34.789118,2.899093,33.067477,2.755623,0,3.301927,8,0,0,0,9,1,1,5,7,9,6,9,0.1106,5,50,1,11,111,51.208628,0.640072,0.235160,0,1


#### Color Type

In [ ]:
combined_df['subcolor_type1'] = combined_df['color_type'].apply(lambda x: x.split()[0])
combined_df['subcolor_type1_enc'] = combined_df['subcolor_type1'].factorize()[0]

combined_df['subcolor_type2'] = combined_df['color_type'].apply(lambda x: 'NONE' if len(x.split())==1 else x.split()[1])
combined_df['subcolor_type2_enc'] = combined_df['subcolor_type2'].factorize()[0]

In [ ]:
combined_df["color_type_num_words"] = combined_df["color_type"].apply(lambda x: len(str(x).split()))
combined_df["color_type_num_chars"] = combined_df["color_type"].apply(lambda x: len(str(x)))

In [ ]:
tfidf = TfidfVectorizer(analyzer='char')
features = tfidf.fit_transform(combined_df.color_type).toarray()
features_df = pd.DataFrame(features, columns=tfidf.get_feature_names())
print(features_df.shape)
combined_df = pd.merge(combined_df, features_df, left_index=True, right_index=True)

(26906, 24)


In [ ]:
countvec = CountVectorizer(analyzer='char')
features = countvec.fit_transform(combined_df.color_type).toarray()
features_df = pd.DataFrame(features, columns=countvec.get_feature_names())
print(features_df.shape)
combined_df = pd.merge(combined_df, features_df, left_index=True, right_index=True)

(26906, 24)


In [ ]:
combined_df.drop(['color_type', 'subcolor_type1', 'subcolor_type2'], axis=1, inplace=True)
combined_df.head()

,condition,length(m),X1,X2,iss_dt_year,iss_dt_quarter,iss_dt_month,iss_dt_week,iss_dt_day_year,iss_dt_day_month,iss_dt_day_week,iss_dt_day_weekend,lst_dt_year,lst_dt_quarter,lst_dt_month,lst_dt_week,lst_dt_day_year,lst_dt_day_month,lst_dt_day_week,lst_dt_day_weekend,lst_dt_hour,lst_dt_minute,lst_dt_minutes_elapsed,dt_diff_days,dt_diff_weeks,dt_diff_mnths,dt_diff_yrs,dt_diff_sec,dt_diff_min,dt_diff_hrs,curr_iss_dt_diff_mnths,curr_iss_dt_diff_yrs,curr_lst_dt_diff_mnths,curr_lst_dt_diff_yrs,condition_odd_even_enc,condition*X1,condition*X2,condition%X1,condition%X2,pet_id_odd_even_enc,...,h_x,i_x,k_x,l_x,m_x,n_x,o_x,p_x,r_x,s_x,t_x,u_x,v_x,w_x,x_x,y_x,_y,a_y,b_y,c_y,d_y,e_y,f_y,g_y,h_y,i_y,k_y,l_y,m_y,n_y,o_y,p_y,r_y,s_y,t_y,u_y,v_y,w_y,x_y,y_y
0,2,0.80,13,9,2016,3,7,27,192,10,6,1,2016,3,9,38,265,21,2,0,16,25,985.0,73.684028,10.526290,2.420880,0.201740,6366300.0,106105.0,1768.416667,49.409542,4.117462,46.988661,3.915722,0,2.962496,18,1,1,0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.276002,0.269639,0.0,0.262714,0.0,0.241936,0.0,0.0,0.276241,0.0,0.356584,1,1,3,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,0,1
1,1,0.72,13,9,2013,4,11,47,325,21,3,0,2018,4,12,52,361,27,3,0,17,47,1067.0,1862.740972,266.105853,61.200139,5.100012,160940820.0,2682347.0,44705.783333,81.015940,6.751328,19.815801,1.651317,1,2.351335,9,0,0,1,...,0.587465,0.469963,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.354338,0.0,0.0,0.404581,0.0,0.000000,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0
2,3,0.15,15,4,2014,3,9,39,271,28,6,1,2016,4,10,42,293,19,2,0,8,24,504.0,752.350000,107.478571,24.718372,2.059864,65003040.0,1083384.0,18056.400000,70.798071,5.899839,46.079699,3.839975,1,3.556893,12,0,1,1,...,0.000000,0.000000,0.000000,0.000000,0.0,0.477268,0.466266,0.0,0.454291,0.0,0.000000,0.0,0.0,0.477682,0.0,0.000000,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0
3,1,0.62,0,1,2016,4,12,52,366,31,5,1,2019,1,1,4,25,25,4,0,18,30,1110.0,755.770833,107.967262,24.830763,2.069230,65298600.0,1088310.0,18138.500000,43.692792,3.641066,18.862029,1.571836,1,0.000000,1,0,0,0,...,0.587465,0.469963,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.354338,0.0,0.0,0.404581,0.0,0.000000,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0
4,2,0.50,18,4,2017,3,9,39,271,28,3,0,2017,4,11,46,323,19,6,1,9,38,578.0,52.401389,7.485913,1.721642,0.143470,4527480.0,75458.0,1257.633333,34.789118,2.899093,33.067477,2.755623,0,3.301927,8,0,0,0,...,0.000000,0.000000,0.560885,0.425915,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0,1,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0


#### X1 and X2

In [ ]:
combined_df['X1_odd_even'] = combined_df['X1'].apply(lambda x: 'EVEN' if x%2==0 else 'ODD')
combined_df['X2_odd_even'] = combined_df['X2'].apply(lambda x: 'EVEN' if x%2==0 else 'ODD')

combined_df['X1_odd_even_enc'] = combined_df['X1_odd_even'].factorize()[0]
combined_df['X2_odd_even_enc'] = combined_df['X2_odd_even'].factorize()[0]

In [ ]:
combined_df['Xavg'] = (combined_df['X1']+combined_df['X2'])/2
combined_df['Xdiff'] = np.abs(combined_df['X1']-combined_df['X2'])
combined_df['Xmul'] = combined_df['X1']*combined_df['X2']
combined_df['Xdiv'] = combined_df.apply(lambda row: 0 if row['X1']==0 or row['X2']==0 else row['X2']/row['X1'], axis=1)
combined_df['Xdiv'] = np.cbrt(combined_df['Xdiv'])
combined_df['Xmod'] = combined_df.apply(lambda row: 0 if row['X1']==0 or row['X2']==0 else row['X1']/row['X2'], axis=1)
combined_df['Xdiag'] = np.sqrt(combined_df['X1']**2 + combined_df['X2']**2)
combined_df['Xcbrt'] = np.cbrt(combined_df['X1']**2 + combined_df['X2']**2)

In [ ]:
combined_df['X1_cat'] = combined_df['X1'].apply(lambda x: '<=6' if x<=6 else '6to13' if x>6 and x<=13 else '>13')
combined_df['X2_cat'] = combined_df['X2'].apply(lambda x: '<=3' if x<=3 else '4to7' if x>3 and x<=7 else '>7')

combined_df['X1_cat_enc'] = combined_df['X1_cat'].factorize()[0]
combined_df['X2_cat_enc'] = combined_df['X2_cat'].factorize()[0]

In [ ]:
combined_df['X1_vs_X2'] = combined_df.apply(lambda row: 'X1=X2' if row['X1']==row['X2'] else 
                                            'X1>X2' if row['X1']>row['X2'] else 
                                            'X1<X2' if row['X1']<row['X2'] else 'NA', axis=1)

combined_df['X1_vs_X2_enc'] = combined_df['X1_vs_X2'].factorize()[0]

In [ ]:
#combined_df['length/X1'] = combined_df.apply(lambda row: 0 if row['length(m)']==0 or row['X1']==0 else row['length(m)']/row['X1'], axis=1)
combined_df['length/X2'] = combined_df.apply(lambda row: 0 if row['length(m)']==0 or row['X2']==0 else row['length(m)']/row['X2'], axis=1)
combined_df['length/Xavg'] = combined_df.apply(lambda row: 0 if row['length(m)']==0 or row['Xavg']==0 else np.log1p(row['length(m)']/row['Xavg']), axis=1)
combined_df['length/Xdiag'] = combined_df.apply(lambda row: 0 if row['length(m)']==0 or row['Xdiag']==0 else np.log1p(row['length(m)']/row['Xdiag']), axis=1)

In [ ]:
#combined_df['height/X1'] = combined_df.apply(lambda row: 0 if row['height(m)']==0 or row['X1']==0 else row['height(m)']/row['X1'], axis=1)
combined_df['height/X2'] = combined_df.apply(lambda row: 0 if row['height(m)']==0 or row['X2']==0 else row['height(m)']/row['X2'], axis=1)
combined_df['height/Xavg'] = combined_df.apply(lambda row: 0 if row['height(m)']==0 or row['Xavg']==0 else np.log1p(row['height(m)']/row['Xavg']), axis=1)
combined_df['height/Xdiag'] = combined_df.apply(lambda row: 0 if row['height(m)']==0 or row['Xdiag']==0 else np.log1p(row['height(m)']/row['Xdiag']), axis=1)

In [ ]:
#combined_df['diag/X1'] = combined_df.apply(lambda row: 0 if row['diag(cm)']==0 or row['X1']==0 else row['diag(cm)']/row['X1'], axis=1)
combined_df['diag/X2'] = combined_df.apply(lambda row: 0 if row['diag(cm)']==0 or row['X2']==0 else row['diag(cm)']/row['X2'], axis=1)
combined_df['diag/Xavg'] = combined_df.apply(lambda row: 0 if row['diag(cm)']==0 or row['Xavg']==0 else np.log1p(row['diag(cm)']/row['Xavg']), axis=1)
combined_df['diag/Xdiag'] = combined_df.apply(lambda row: 0 if row['diag(cm)']==0 or row['Xdiag']==0 else np.log1p(row['diag(cm)']/row['Xdiag']), axis=1)

In [ ]:
#combined_df['area/X1'] = combined_df.apply(lambda row: 0 if row['area(m)']==0 or row['X1']==0 else row['area(m)']/row['X1'], axis=1)
combined_df['area/X2'] = combined_df.apply(lambda row: 0 if row['area(m)']==0 or row['X2']==0 else row['area(m)']/row['X2'], axis=1)
combined_df['area/Xavg'] = combined_df.apply(lambda row: 0 if row['area(m)']==0 or row['Xavg']==0 else np.log1p(row['area(m)']/row['Xavg']), axis=1)
combined_df['area/Xdiag'] = combined_df.apply(lambda row: 0 if row['area(m)']==0 or row['Xdiag']==0 else np.log1p(row['area(m)']/row['Xdiag']), axis=1)

In [ ]:
combined_df.drop(['X1_odd_even', 'X2_odd_even', 'X1_cat', 'X2_cat', 'X1_vs_X2'], axis=1, inplace=True)
combined_df.head()

,condition,length(m),X1,X2,iss_dt_year,iss_dt_quarter,iss_dt_month,iss_dt_week,iss_dt_day_year,iss_dt_day_month,iss_dt_day_week,iss_dt_day_weekend,lst_dt_year,lst_dt_quarter,lst_dt_month,lst_dt_week,lst_dt_day_year,lst_dt_day_month,lst_dt_day_week,lst_dt_day_weekend,lst_dt_hour,lst_dt_minute,lst_dt_minutes_elapsed,dt_diff_days,dt_diff_weeks,dt_diff_mnths,dt_diff_yrs,dt_diff_sec,dt_diff_min,dt_diff_hrs,curr_iss_dt_diff_mnths,curr_iss_dt_diff_yrs,curr_lst_dt_diff_mnths,curr_lst_dt_diff_yrs,condition_odd_even_enc,condition*X1,condition*X2,condition%X1,condition%X2,pet_id_odd_even_enc,...,h_y,i_y,k_y,l_y,m_y,n_y,o_y,p_y,r_y,s_y,t_y,u_y,v_y,w_y,x_y,y_y,X1_odd_even_enc,X2_odd_even_enc,Xavg,Xdiff,Xmul,Xdiv,Xmod,Xdiag,Xcbrt,X1_cat_enc,X2_cat_enc,X1_vs_X2_enc,length/X2,length/Xavg,length/Xdiag,height/X2,height/Xavg,height/Xdiag,diag/X2,diag/Xavg,diag/Xdiag,area/X2,area/Xavg,area/Xdiag
0,2,0.80,13,9,2016,3,7,27,192,10,6,1,2016,3,9,38,265,21,2,0,16,25,985.0,73.684028,10.526290,2.420880,0.201740,6366300.0,106105.0,1768.416667,49.409542,4.117462,46.988661,3.915722,0,2.962496,18,1,1,0,...,0,0,0,0,0,1,1,0,1,0,1,0,0,1,0,1,0,0,11.0,4,117,0.884640,1.444444,15.811388,6.299605,0,0,0,0.088889,0.070204,0.049358,0.008644,0.007048,0.004908,8.930824,2.117103,1.805582,0.027720,0.022427,0.015655
1,1,0.72,13,9,2013,4,11,47,325,21,3,0,2018,4,12,52,361,27,3,0,17,47,1067.0,1862.740972,266.105853,61.200139,5.100012,160940820.0,2682347.0,44705.783333,81.015940,6.751328,19.815801,1.651317,1,2.351335,9,0,0,1,...,1,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,11.0,4,117,0.884640,1.444444,15.811388,6.299605,0,0,0,0.080000,0.063402,0.044530,0.015767,0.012818,0.008935,8.153887,2.037494,1.730110,0.035515,0.028644,0.020014
2,3,0.15,15,4,2014,3,9,39,271,28,6,1,2016,4,10,42,293,19,2,0,8,24,504.0,752.350000,107.478571,24.718372,2.059864,65003040.0,1083384.0,18056.400000,70.798071,5.899839,46.079699,3.839975,1,3.556893,12,0,1,1,...,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,1,9.5,11,60,0.643660,3.750000,15.524175,6.223084,1,1,0,0.037500,0.015666,0.009616,0.102250,0.042152,0.026005,10.890965,1.720204,1.336630,0.061922,0.025738,0.015829
3,1,0.62,0,1,2016,4,12,52,366,31,5,1,2019,1,1,4,25,25,4,0,18,30,1110.0,755.770833,107.967262,24.830763,2.069230,65298600.0,1088310.0,18138.500000,43.692792,3.641066,18.862029,1.571836,1,0.000000,1,0,0,0,...,1,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0.5,1,0,0.000000,0.000000,1.000000,1.000000,2,2,1,0.620000,0.806476,0.482426,0.178200,0.304834,0.163988,64.510095,4.867690,4.182204,0.332391,0.509695,0.286975
4,2,0.50,18,4,2017,3,9,39,271,28,3,0,2017,4,11,46,323,19,6,1,9,38,578.0,52.401389,7.485913,1.721642,0.143470,4527480.0,75458.0,1257.633333,34.789118,2.899093,33.067477,2.755623,0,3.301927,8,0,0,0,...,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,11.0,14,72,0.605707,4.500000,18.439089,6.979532,1,1,0,0.125000,0.044452,0.026755,0.027650,0.010004,0.005980,12.802157,1.732598,1.328977,0.058790,0.021153,0.012673


### Define categorical columns

In [ ]:
categorical_columns = ['condition','X1','X2','condition_odd_even_enc','pet_id_odd_even_enc','petid%X1','petid%X2','petid%condition','l_vs_h_enc','pet_size_enc','subcolor_type1_enc','subcolor_type2_enc','X1_odd_even_enc','X2_odd_even_enc','X1_cat_enc','X2_cat_enc','X1_vs_X2_enc']
categorical_columns_indices = [combined_df.columns.get_loc(col) for col in categorical_columns]
print(categorical_columns_indices)

[0, 2, 3, 34, 39, 40, 41, 42, 57, 58, 59, 60, 111, 112, 120, 121, 122]


In [ ]:
combined_df[categorical_columns].head()

,condition,X1,X2,condition_odd_even_enc,pet_id_odd_even_enc,petid%X1,petid%X2,petid%condition,l_vs_h_enc,pet_size_enc,subcolor_type1_enc,subcolor_type2_enc,X1_odd_even_enc,X2_odd_even_enc,X1_cat_enc,X2_cat_enc,X1_vs_X2_enc
0,2,13,9,0,0,2,0,1,0,0,0,0,0,0,0,0,0
1,1,13,9,1,1,7,4,0,0,1,1,1,0,0,0,0,0
2,3,15,4,1,1,0,2,0,1,1,0,1,0,1,1,1,0
3,1,0,1,1,0,0,0,0,0,1,1,1,1,0,2,2,1
4,2,18,4,0,0,9,1,1,0,1,2,1,1,1,1,1,0


### Separate train/predict datasets

In [ ]:
# Segregate combined_df into train/predict datasets
train_x = combined_df[:18834]
predict_x = combined_df[18834:]

print("train_x: {}".format(train_x.shape))
print("predict_x: {}".format(predict_x.shape))

train_x: (18834, 135)
predict_x: (8072, 135)


In [ ]:
train_x_pc = train_x.copy()
Xpredict = predict_x.copy()

### Split training data into train/ validation/ test datasets

In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=10)
for train_index, test_index in sss.split(train_x_pc.values, train_y_pc.values):
    Xtrain_pc, Xvalidation_pc = train_x_pc.iloc[train_index], train_x_pc.iloc[test_index]
    Ytrain_pc, Yvalidation_pc = train_y_pc.iloc[train_index], train_y_pc.iloc[test_index]

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=20)
for train_index, test_index in sss.split(Xvalidation_pc.values, Yvalidation_pc.values):
    Xvalidation_pc, Xtest_pc = Xvalidation_pc.iloc[train_index], Xvalidation_pc.iloc[test_index]
    Yvalidation_pc, Ytest_pc = Yvalidation_pc.iloc[train_index], Yvalidation_pc.iloc[test_index]

print("------------------------- Training Dataset -------------------------")
print("Xtrain_pc shape: {}".format(Xtrain_pc.shape))
print("Ytrain_pc shape: {}".format(Ytrain_pc.shape))

print("\n------------------------- Validation Dataset -------------------------")
print("Xvalidation_pc shape: {}".format(Xvalidation_pc.shape))
print("Yvalidation_pc shape: {}".format(Yvalidation_pc.shape))

print("\n------------------------- Test Dataset -------------------------")
print("Xtest_pc shape: {}".format(Xtest_pc.shape))
print("Ytest_pc shape: {}".format(Ytest_pc.shape))

print("\n------------------------- Prediction Dataset -------------------------")
print("Xpredict shape: {}".format(Xpredict.shape))

------------------------- Training Dataset -------------------------
Xtrain_pc_full shape: (42238, 135)
Ytrain_pc_full shape: (42238, 1)
Xtrain_pc shape: (31678, 135)
Ytrain_pc shape: (31678, 1)

------------------------- Validation Dataset -------------------------
Xvalidation_pc shape: (5280, 135)
Yvalidation_pc shape: (5280, 1)

------------------------- Test Dataset -------------------------
Xtest_pc shape: (5280, 135)
Ytest_pc shape: (5280, 1)

------------------------- Prediction Dataset -------------------------
Xpredict shape: (8072, 135)


### Handling class imbalance

In [ ]:
#sm = SMOTE()
sm = ADASYN()
sm_x, sm_y = sm.fit_sample(Xtrain_pc, Ytrain_pc.ravel())
Xtrain_pc = sm_x
Ytrain_pc = np.array([sm_y]).T
print("Class balancing done.")
print("Xtrain_pc shape: {}".format(Xtrain_pc.shape))
print("Ytrain_pc shape: {}".format(Ytrain_pc.shape))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.



Class balancing done.
train_x_pc shape: (42238, 135)
train_y_pc shape: (42238, 1)


In [ ]:
Xtrain_pc = pd.DataFrame(Xtrain_pc, columns=train_x.columns)
Xtrain_pc[categorical_columns] = Xtrain_pc[categorical_columns].astype(int)
Xtrain_pc[categorical_columns].head()

,condition,X1,X2,condition_odd_even_enc,pet_id_odd_even_enc,petid%X1,petid%X2,petid%condition,l_vs_h_enc,pet_size_enc,subcolor_type1_enc,subcolor_type2_enc,X1_odd_even_enc,X2_odd_even_enc,X1_cat_enc,X2_cat_enc,X1_vs_X2_enc
0,2,13,9,0,0,2,0,1,0,0,0,0,0,0,0,0,0
1,1,13,9,1,1,7,4,0,0,1,1,1,0,0,0,0,0
2,3,15,4,1,1,0,2,0,1,1,0,1,0,1,1,1,0
3,1,0,1,1,0,0,0,0,0,1,1,1,1,0,2,2,1
4,2,18,4,0,0,9,1,1,0,1,2,1,1,1,1,1,0


In [ ]:
Ytrain_pc = pd.DataFrame(Ytrain_pc, columns=['Class'])

tmp = Yvalidation_pc.values
Yvalidation_pc = pd.DataFrame(tmp, columns=['Class'])

tmp = Ytest_pc.values
Ytest_pc = pd.DataFrame(tmp, columns=['Class'])

In [ ]:
temp_df = pd.DataFrame(Ytrain_pc.values, columns=['Class'])
temp_df.groupby(['Class']).size().reset_index().rename(columns={0:'count'})

In [ ]:
temp_df = pd.DataFrame(Yvalidation_pc.values, columns=['Class'])
temp_df.groupby(['Class']).size().reset_index().rename(columns={0:'count'})

In [ ]:
temp_df = pd.DataFrame(Ytest_pc.values, columns=['Class'])
temp_df.groupby(['Class']).size().reset_index().rename(columns={0:'count'})

## Pet Category Prediction

### Hyperparameter search using Optuna

In [ ]:
def objective(trial):
    
    model = CatBoostClassifier(
        objective='MultiClass',
        eval_metric='Accuracy',
        num_boost_round=5000,
        learning_rate=trial.suggest_loguniform("learning_rate", 1e-2, 1e-1),
        reg_lambda=trial.suggest_loguniform("reg_lambda", 1e-4, 1.0),
        bootstrap_type='Poisson',
        subsample=trial.suggest_uniform("subsample", 0.5, 1.0),
        max_depth=trial.suggest_int("max_depth", 6, 12), 
        grow_policy='Lossguide',
        min_data_in_leaf=trial.suggest_int("min_data_in_leaf", 1, 15), 
        max_leaves=trial.suggest_int("max_leaves", 35, 200),
        task_type='GPU',
        verbose=0
    )

    model.fit(Xtrain_pc, Ytrain_pc, eval_set=[(Xvalidation_pc, Yvalidation_pc)], 
              cat_features=categorical_columns_indices,
              early_stopping_rounds=200, verbose=False)
    y_pred = model.predict(Xtest_pc)
    y_pred = y_pred.astype(int)
    f1 = f1_score(Ytest_pc, y_pred, average='weighted')

    return f1

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=250)

[I 2020-08-21 21:00:21,813] Trial 0 finished with value: 0.9577999269061988 and parameters: {'learning_rate': 0.059980834933015266, 'reg_lambda': 0.006826139227884132, 'subsample': 0.5946948759182077, 'max_depth': 11, 'min_data_in_leaf': 13, 'max_leaves': 58}. Best is trial 0 with value: 0.9577999269061988.
[I 2020-08-21 21:02:22,880] Trial 1 finished with value: 0.9577955706024224 and parameters: {'learning_rate': 0.02102007117864995, 'reg_lambda': 0.00017406266916857147, 'subsample': 0.8913138224557675, 'max_depth': 8, 'min_data_in_leaf': 9, 'max_leaves': 182}. Best is trial 0 with value: 0.9577999269061988.
[I 2020-08-21 21:03:24,832] Trial 2 finished with value: 0.9585602156605871 and parameters: {'learning_rate': 0.032866817826021306, 'reg_lambda': 0.0002799387788023879, 'subsample': 0.9305862236532063, 'max_depth': 7, 'min_data_in_leaf': 10, 'max_leaves': 167}. Best is trial 2 with value: 0.9585602156605871.
[I 2020-08-21 21:04:12,607] Trial 3 finished with value: 0.9589669983563

KeyboardInterrupt: ignored

In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("Value: {}".format(trial.value))

print("Params: ")
for key, value in trial.params.items():
    print(" {}: {}".format(key, value))

Number of finished trials: 27
Best trial:
Value: 0.9595271942422664
Params: 
 learning_rate: 0.05457686888559569
 reg_lambda: 0.00014589161761291262
 subsample: 0.9105553766697733
 max_depth: 6
 min_data_in_leaf: 11
 max_leaves: 116


### Build the model for pet-category prediction

In [ ]:
# Define K-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=20)
y_pred = 0

In [ ]:
Xtrain_tmp = pd.concat([Xtrain_pc, Xvalidation_pc], axis=0)
Ytrain_tmp = pd.concat([Ytrain_pc, Yvalidation_pc], axis=0)

In [ ]:
# Train the model using K-fold
counter = 0

for train, val in kfold.split(Xtrain_tmp.values, Ytrain_tmp.values):
    counter += 1

    train_x, train_y = Xtrain_tmp.iloc[train], Ytrain_tmp.iloc[train]
    val_x, val_y = Xtrain_tmp.iloc[val], Ytrain_tmp.iloc[val]

    model = CatBoostClassifier(
        objective='MultiClass',
        eval_metric='Accuracy',
        num_boost_round=5000,
        learning_rate=0.0546,
        reg_lambda=0.000146,
        bootstrap_type='Poisson',
        subsample=0.911,
        max_depth=6, 
        grow_policy='Lossguide',
        min_data_in_leaf=11, 
        max_leaves=116,
        task_type='GPU',
        verbose=0
    )

    model.fit(train_x, train_y, eval_set=[(val_x, val_y)], 
              cat_features=categorical_columns_indices,
              early_stopping_rounds=200, verbose=100)
    pred = model.predict(Xtest_pc)

    y_pred += pred

y_pred = y_pred/float(counter)

0:	learn: 0.7944501	test: 0.7819264	best: 0.7819264 (0)	total: 49.3ms	remaining: 4m 6s
100:	learn: 0.9533702	test: 0.9418290	best: 0.9418290 (100)	total: 4.58s	remaining: 3m 42s
200:	learn: 0.9676508	test: 0.9480519	best: 0.9480519 (197)	total: 8.4s	remaining: 3m 20s
300:	learn: 0.9764296	test: 0.9504870	best: 0.9507576 (291)	total: 12.2s	remaining: 3m 11s
400:	learn: 0.9837653	test: 0.9515693	best: 0.9523810 (332)	total: 16s	remaining: 3m 3s
500:	learn: 0.9880043	test: 0.9542749	best: 0.9545455 (482)	total: 19.8s	remaining: 2m 57s
600:	learn: 0.9927244	test: 0.9561688	best: 0.9561688 (598)	total: 23.7s	remaining: 2m 53s
700:	learn: 0.9957609	test: 0.9567100	best: 0.9567100 (630)	total: 27.9s	remaining: 2m 50s
800:	learn: 0.9978053	test: 0.9569805	best: 0.9572511 (731)	total: 31.9s	remaining: 2m 47s
900:	learn: 0.9989778	test: 0.9564394	best: 0.9572511 (731)	total: 35.8s	remaining: 2m 43s
bestTest = 0.9572510823
bestIteration = 731
Shrink model to first 732 iterations.
0:	learn: 0.7946

### Validate the model

In [ ]:
# Print log_loss, accuracy and F1-score
y_pred = y_pred.astype(int)
f1 = f1_score(Ytest_pc, y_pred, average='weighted')
print('Overall F1-Score:', f1)

Overall F1-Score: 0.9537586639407394


In [ ]:
print(classification_report(Ytest_pc, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1325
           1       0.91      0.93      0.92      1286
           2       0.91      0.91      0.91      1328
           3       1.00      0.98      0.99      1341

    accuracy                           0.95      5280
   macro avg       0.95      0.95      0.95      5280
weighted avg       0.95      0.95      0.95      5280



## Train model on entire data

In [ ]:
# Define K-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=30)
y_pred = 0

In [ ]:
Xtrain_tmp = pd.concat([Xtrain_pc, Xvalidation_pc, Xtest_pc], axis=0)
Ytrain_tmp = pd.concat([Ytrain_pc, Yvalidation_pc, Ytest_pc], axis=0)

In [ ]:
# Train the model using K-fold
counter = 0

for train, val in kfold.split(Xtrain_tmp.values, Ytrain_tmp.values):
    counter += 1

    train_x, train_y = Xtrain_tmp.iloc[train], Ytrain_tmp.iloc[train]
    val_x, val_y = Xtrain_tmp.iloc[val], Ytrain_tmp.iloc[val]

    model = CatBoostClassifier(
        objective='MultiClass',
        eval_metric='Accuracy',
        num_boost_round=5000,
        learning_rate=0.0546,
        reg_lambda=0.000146,
        bootstrap_type='Poisson',
        subsample=0.911,
        max_depth=6, 
        grow_policy='Lossguide',
        min_data_in_leaf=11, 
        max_leaves=116,
        task_type='GPU',
        verbose=0
    )

    model.fit(train_x, train_y, eval_set=[(val_x, val_y)], 
              cat_features=categorical_columns_indices,
              early_stopping_rounds=200, verbose=100)
    pred = model.predict(Xpredict)

    y_pred += pred

y_pred = y_pred/float(counter)

0:	learn: 0.7867628	test: 0.7826705	best: 0.7826705 (0)	total: 52.4ms	remaining: 4m 21s
100:	learn: 0.9525438	test: 0.9514678	best: 0.9514678 (98)	total: 5.06s	remaining: 4m 5s
200:	learn: 0.9661704	test: 0.9578598	best: 0.9585701 (166)	total: 9.2s	remaining: 3m 39s
300:	learn: 0.9752986	test: 0.9607008	best: 0.9614110 (298)	total: 13.3s	remaining: 3m 27s
400:	learn: 0.9820329	test: 0.9623580	best: 0.9623580 (399)	total: 17.6s	remaining: 3m 22s
500:	learn: 0.9877151	test: 0.9609375	best: 0.9623580 (399)	total: 21.7s	remaining: 3m 14s
bestTest = 0.9623579545
bestIteration = 399
Shrink model to first 400 iterations.
0:	learn: 0.7940233	test: 0.7999527	best: 0.7999527 (0)	total: 37.6ms	remaining: 3m 7s
100:	learn: 0.9518598	test: 0.9533617	best: 0.9533617 (100)	total: 4.29s	remaining: 3m 27s
200:	learn: 0.9661441	test: 0.9569129	best: 0.9573864 (185)	total: 8.19s	remaining: 3m 15s
300:	learn: 0.9747461	test: 0.9592803	best: 0.9592803 (295)	total: 12.1s	remaining: 3m 8s
400:	learn: 0.98137

## Create submission file

In [ ]:
y_pred = y_pred.astype(int)
predict_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Adopt a Buddy/Dataset/test.csv")
submit_df = pd.DataFrame()
submit_df['pet_id'] = predict_df['pet_id']
submit_df['pet_category'] = y_pred
submit_df['pet_category'] = submit_df['pet_category'].apply(lambda x: 4 if x==3 else x)
submit_df.head()

,pet_id,pet_category
0,ANSL_75005,2
1,ANSL_76663,1
2,ANSL_58259,2
3,ANSL_67171,2
4,ANSL_72871,2


In [ ]:
submit_df.to_csv("/content/drive/My Drive/Colab Notebooks/Adopt a Buddy/Predictions/predictions_v56_pc_CB.csv", index=False)